In [114]:
#import needed package
import asyncio
import httpx
import pandas as pd
import secret
import refresh
import requests
from time import perf_counter

In [115]:
#authtokens and headers
id_token = refresh.refreshed()
headers = {"content-type":"application/json", "x-api-key": secret.api_key, "id-token": id_token}

In [116]:
#scrape all
vinny_id = 62869

In [117]:
#api calls
sem = asyncio.Semaphore(100)
#scrape all id

async def get_result(vinny_id:int):
    async with sem:
        async with httpx.AsyncClient() as client:
            r = await client.get(f'https://bps-world.vincere.io/api/v2/candidate/{vinny_id}/industries', headers=headers)
            if r.status_code == 200:
                data = [{**d, 'vinny_id': vinny_id} for d in r.json()]
                results.append(data)
                vincere.append(vinny_id)
                return
            else:
                return

#trigger main function
async def main():
    tasks = []
    vinny_id = 62869
    #gen_id = df['vinny_id']
    while vinny_id <= 103739:
        tasks.append(get_result(vinny_id))
        vinny_id += 1
#     tasks = []
#     vinny_id = 85738
#     tasks.append(get_result(vinny_id))
    await asyncio.gather(*tasks, return_exceptions=True)

#future container for finding errors and stuff
results = []

In [118]:
#trigger the whole function
start = perf_counter()
await main()
stop = perf_counter()
print('time taken:', stop - start, ' seconds')

time taken: 1047.8320414999998  seconds


In [105]:
results

[[{'id': 28966, 'name': 'Engineering & Construction', 'vinny_id': 85738},
  {'id': 28994, 'name': 'Tech Consulting & Services', 'vinny_id': 85738},
  {'id': 28890, 'name': 'Technology', 'vinny_id': 85738}],
 [{'id': '123123', 'name': 'test', 'vinny_id': 123123}]]

In [119]:
flat_data = [d for sublist in results for d in sublist]

In [120]:
df = pd.DataFrame(flat_data, columns=['id', 'name', 'vinny_id'])

In [121]:
df

,id,name,vinny_id
0,28776,Finance & Accounting,62956
1,28882,Engineering,62873
2,28776,Finance & Accounting,62896
3,28868,Sales & Marketing,62909
4,28868,Sales & Marketing,62874
...,...,...,...
37277,28890,Technology,103736
37278,28890,Technology,103734
37279,28890,Technology,103720
37280,28776,Finance & Accounting,103738


In [122]:
df.to_excel('vincere_industry_candidate_230223.xlsx')